# iGEM 2021
## Saccharomyces cerevisiae GSM analysis
#### Groningen and DTU BioBuilders teams collaboration

In [3]:
# import necessary libraries, especially cobra package, specialized on GSM
import numpy as np
import matplotlib.pyplot as plt
import cobra

# model originally from:
# Mo ML, Palsson BO, Herrgård MJ. Connecting extracellular metabolomic measurements to intracellular flux states in yeast. BMC Syst Biol. 2009 Mar 25;3:37. doi: 10.1186/1752-0509-3-37. PMID: 19321003; PMCID: PMC2679711.

# load model
saccerv = cobra.io.read_sbml_model("./data/iMM904.xml")

First of all, we will do a general analysis of some basic concepts from the model. This is something that could also perfectly be done by manual checking on the sbml file, but cobrapy allows some very simple funcionalities.

A very important thing on GSM is to keep track on where in the cells the different reactions are ocurring, and that is why is nice to know which compartments are defined in the model and how are they referred as.

In [79]:
saccerv.compartments

{'c': 'cytosol',
 'e': 'extracellular space',
 'm': 'mitochondria',
 'x': 'peroxisome/glyoxysome',
 'r': 'endoplasmic reticulum',
 'v': 'vacuole',
 'g': 'golgi apparatus',
 'n': 'nucleus'}

After that, is nice to see which conditions are defined for our medium. Usually GSM is based on constrainin reactions with your knowledge from literature or experiments done, and for cases you have no data, you just do not include any constrain, allowing a complete freedom on that dimension/reaction.

This is what happens in the next part, we see the medium comes defined by some exchange reactions ("EX_"), where the limits of the cell uptake for different metabolites are placed.

In [80]:
# It is seen that the two bounds are constrained for 'EX_glc__D_e' and 'EX_o2_e', so glucose and oxygen, respectively.
saccerv.medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_so4_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0}

In [81]:
# Let's try first to check the glucose reaction, as we see that the glucose uptake is limited to 10 mmol/gDW h^-1, which is
# the usual units for GSM
saccerv.reactions.EX_glc__D_e

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x07f34a030cb80
Stoichiometry,glc__D_e <=> D-Glucose <=>
GPR,
Lower bound,-10.0
Upper bound,999999.0


In [82]:
# but this is not the reaction that introduces glucose to the cell, it is just the external exchange for placing the constrains,
# so let's find in which reactions is this metabolite involved
saccerv.metabolites.glc__D_e

Metabolite identifier,glc__D_e
Name,D-Glucose
Memory address,0x07f34a2883670
Formula,C6H12O6
Compartment,e
In 4 reaction(s),"EX_glc__D_e, SUCRe, GLCt1, 13BGHe"


In [83]:
# as seen in the previous summary, glc__D_e metabolite is involved in 4 reactions, being GLCt1 the one for uptake
saccerv.reactions.GLCt1

Reaction identifier,GLCt1
Name,Glucose transport uniport
Memory address,0x07f34a010eac0
Stoichiometry,glc__D_e --> glc__D_c D-Glucose --> D-Glucose
GPR,YHR092C or YLR081W or YOL156W or YDR536W or YHR094C or YEL069C or YDL245C or YJR158W or YFL011W...
Lower bound,0.0
Upper bound,999999.0


And the same would go with oxygen, limited to an uptake of $2 mmol/gDW h^{-1}$

Now, we analyze what this model is trying to optimize by default (meaning that cobrapy allows the user to choose any reaction to optimize).

In [84]:
saccerv.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,1.611,0,0.00%
o2_e,EX_o2_e,2,0,0.00%
pi_e,EX_pi_e,0.05691,0,0.00%
so4_e,EX_so4_e,0.02225,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-18.02,1,36.29%
etoh_e,EX_etoh_e,-15.82,2,63.70%
for_e,EX_for_e,-0.001488,1,0.00%
h2o_e,EX_h2o_e,-5.64,0,0.00%


As usual, by default it optimizes biomass production, and here we can see a summary where after the optimization, a biomass production of $0.288 mmol/gDW h^{-1}$ is obtained. Besides, it can be seen the top 5 uptake and secretion metabolites with their flux.

But now, let's go to the fun part and include our synthetic biology, for the production of our introduced genes. In this case, we will try introducing one of each of the genes at a time to produce the different amylases. This requires to specify the expression of dna, rna and amino acid sequence --> protein.

As the sequences are long, we calculate the stoichiometry with a script which is on the github repo and imported here
based on the logic behind introduction of heterologous protein production in the paper:
*Irani ZA, Kerkhoven EJ, Shojaosadati SA, Nielsen J (2016) Genome-scale metabolic model of Pichia pastoris with native and humanized glycosylation of recombinant proteins. Biotechnol Bioeng 113:961–969. doi: 10.1002/bit.25863.*

In [4]:
# import our code with one method for transcription and one for introduction of correct stoichiometry into the model for protein expression 
import stoichiometry_groningen as stoich 
from Groningen_sequences import * # all sequences for dna and protein are saved in one python file as strings

In [5]:
# the transcription method allows transcription when a DNA sequence is introduced as parameter
A_oryzae_rna = stoich.transcription(A_oryzae_dna)
B_amyloliquenifaciens_rna = stoich.transcription(B_amyloliquenifaciens_dna)
B_licheniformis_rna = stoich.transcription(B_licheniformis_dna)
B_Subtilis_rna = stoich.transcription(B_Subtilis_dna)

In [6]:
def add_protein_reaction(model, reaction, lb, ub, seq, seq_type, protein_name = 'amylase'):
    
    stoichiometry = stoich.get_stoichiometry(seq, seq_type, protein_name)
    
    
    reaction = cobra.Reaction(
                reaction,
                name = reaction,
                lower_bound = lb,
                upper_bound = ub
        )
    
    for molecule in stoichiometry:
        reaction.add_metabolites(
            {
                getattr(model.metabolites, molecule): stoichiometry[molecule] 
            }
        )
    
    model.add_reactions([reaction])

# and after that we need to remove them before including the ones for new organism --> create a function so we just call it
def remove_amylase(model):
    model.remove_reactions([model.reactions.amylase_DNA_reaction,
                            model.reactions.amylase_RNA_reaction,
                            model.reactions.amylase_AA_reaction])

In [7]:
# before introducing reactions into the model, we need to define the metabolites involved
# in this case, as for all cases will be amylase, just changing the organism from which we take the gene, we will use a general name

amylase_DNA = cobra.Metabolite(
    'amylase_DNA',
    name='amylase_DNA',
    compartment='c')

amylase_RNA = cobra.Metabolite(
    'amylase_RNA',
    name='amylase_RNA',
    compartment='c')

amylase_AA = cobra.Metabolite(
    'amylase_AA',
    name='amylase_AA',
    compartment='c')

saccerv.add_metabolites([amylase_DNA,
                       amylase_RNA,
                       amylase_AA])

### A. Oryzae

In [24]:
# 0.005-(0.005*(1-0.5)^(1/5)) this is the amount of protein degraded at a typical half-life of 5 hours
# starting at 0.05g/gDW = 10% of total protein as about 50% of gDW is protein
# = ca. 0.00065
# higher values yield an infeasible solution

add_protein_reaction(saccerv, 'amylase_DNA_reaction', 0, 100, A_oryzae_dna, 'dna', 'amylase')
add_protein_reaction(saccerv, 'amylase_RNA_reaction', 0, 100, A_oryzae_rna, 'rna', 'amylase')
add_protein_reaction(saccerv, 'amylase_AA_reaction', 0.00647, 0.00647, A_oryzae_protein, 'aa', 'amylase')

In [26]:
cobra.io.write_sbml_model(saccerv, "./data/test.xml")

In [25]:
A_oryzae = saccerv.optimize().objective_value
saccerv.summary()

Infeasible: None (infeasible).

In [23]:
remove_amylase(saccerv)

### B. Amyloliquenifaciens

In [34]:
add_protein_reaction(saccerv, 'amylase_DNA_reaction', 0, 100, B_amyloliquenifaciens_dna, 'dna', 'amylase')
add_protein_reaction(saccerv, 'amylase_RNA_reaction', 0, 100, B_amyloliquenifaciens_rna, 'rna', 'amylase')
add_protein_reaction(saccerv, 'amylase_AA_reaction', 0.00065, 0.00065, B_amyloliquenifaciens_protein, 'aa', 'amylase')
B_amyloliquenifaciens = saccerv.optimize().objective_value
saccerv.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,1.612,0,0.00%
o2_e,EX_o2_e,2,0,0.00%
pi_e,EX_pi_e,0.0597,0,0.00%
so4_e,EX_so4_e,0.02225,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-18.02,1,36.29%
etoh_e,EX_etoh_e,-15.82,2,63.70%
for_e,EX_for_e,-0.001488,1,0.00%
h2o_e,EX_h2o_e,-5.643,0,0.00%


In [35]:
remove_amylase(saccerv)

### B. Licheniformis

In [36]:
add_protein_reaction(saccerv, 'amylase_DNA_reaction', 0.00005, 0.00005, B_licheniformis_dna, 'dna', 'amylase')
add_protein_reaction(saccerv, 'amylase_RNA_reaction', 0.00005, 0.00005, B_licheniformis_rna, 'rna', 'amylase')
add_protein_reaction(saccerv, 'amylase_AA_reaction', 0.00005, 0.00005, B_licheniformis_protein, 'aa', 'amylase')
B_licheniformis = saccerv.optimize().objective_value
saccerv.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,1.612,0,0.00%
o2_e,EX_o2_e,2,0,0.00%
pi_e,EX_pi_e,0.05969,0,0.00%
so4_e,EX_so4_e,0.02225,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-18.02,1,36.29%
etoh_e,EX_etoh_e,-15.82,2,63.70%
for_e,EX_for_e,-0.001488,1,0.00%
h2o_e,EX_h2o_e,-5.643,0,0.00%


In [37]:
remove_amylase(saccerv)

### B. Subtilis

In [40]:
add_protein_reaction(saccerv, 'amylase_DNA_reaction', 0, 100, B_Subtilis_dna, 'dna', 'amylase')
add_protein_reaction(saccerv, 'amylase_RNA_reaction', 0, 100, B_Subtilis_rna, 'rna', 'amylase')
add_protein_reaction(saccerv, 'amylase_AA_reaction', 0.00065, 0.00065, B_subtilis_protein, 'aa', 'amylase')
B_Subtilis = saccerv.optimize().objective_value
saccerv.summary()

Ignoring reaction 'amylase_DNA_reaction' since it already exists.
Ignoring reaction 'amylase_RNA_reaction' since it already exists.


Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,1.612,0,0.00%
o2_e,EX_o2_e,2,0,0.00%
pi_e,EX_pi_e,0.05975,0,0.00%
so4_e,EX_so4_e,0.02225,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-18.02,1,36.29%
etoh_e,EX_etoh_e,-15.82,2,63.70%
for_e,EX_for_e,-0.001488,1,0.00%
h2o_e,EX_h2o_e,-5.643,0,0.00%
